### 代码流程：
yolov5画框框
先读取标签 -> 标签转换为xyxy -> 画个透明框(输入图像，标签) -> 输出，透明框作为GT

In [1]:
# 画个透明框，xyxy，color
# 将处理后的视频保存为output.avi文件
import cv2  # OpenCV
import numpy as np
import os
# import matplotlib.pyplot as plt
import pandas as pd


# 透明框
def drawTransparentRectangle(file_path, save_path, ob_list, label_name, color = (255,255,255), style = -1):
    img = cv2.imread(file_path)
    res = img
    for ob in ob_list:
        zeros = np.zeros((img.shape), dtype = np.uint8)
        cv2.rectangle(zeros, (ob[0],ob[1]), (ob[2], ob[3]), color, style)
        
        res = res + zeros*0.4
        res[res>255] = 255
        cv2.putText(res, ob[4], (ob[0],ob[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)
    # print(ob)
    cv2.imwrite(save_path, res)


# 有框
def drawDefaultRectangle(file_path, save_path, ob_list, label_name, color = (255,0,0), style = 1):
    img = cv2.imread(file_path)
    res = img
    for ob in ob_list:
        cv2.rectangle(res, (ob[0],ob[1]), (ob[2], ob[3]), color, style)
        cv2.putText(res, ob[4], (ob[2],ob[3]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)
    # print(ob)
    cv2.imwrite(save_path, res)

    
# yolov5的框xyxy
def draw_from_label(img_path, label_path, save_path, label_name, img_width = 1920, img_height = 1080, draw_type = drawTransparentRectangle):
    des_rec = []
    with open(label_path, "r") as f:
        for i in f.readlines():
            temp = i.split(" ")
            label = label_name[int(temp[0])]
            tempx_min = int((float(temp[1]) - 0.5*float(temp[3])) * img_width)
            tempx_max = int((float(temp[1]) + 0.5*float(temp[3])) * img_width)
            tempy_min = int((float(temp[2]) - 0.5*float(temp[4])) * img_height)
            tempy_max = int((float(temp[2]) + 0.5*float(temp[4])) * img_height)
            des_rec.append([tempx_min, tempy_min, tempx_max, tempy_max, label])
            
        draw_type(img_path, save_path, des_rec, label)
            
            


In [2]:
# def add(a,b):
#     return a+b

# def do(a,b,c = add):
#     return c(a,b)

# do(5,2,add)

In [7]:
if __name__ == "__main__":
    img_path = r"E:\yolov5-7.0\runs\val\ggwp4\wrong_pic"
    label_path = r"E:\yolov5-7.0\runs\val\ggwp4\wrong_label"
    ## ground truth
    gt_path = r"E:\yolov5-7.0\runs\val\ggwp4\gt_label"
    save_path = r"E:\yolov5-7.0\runs\val\ggwp4\wrong_label_pic"
    label_name_path = r"z:\datasets\交通灯标签2.txt"
    label_namePD = pd.read_csv(label_name_path, header=None)
    label_name = list(label_namePD[0])

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    ed = 2
    img_list = os.listdir(img_path)
    for img in img_list:
        
        lb = img.replace("jpg","txt") if img.endswith(".jpg") else img.replace("png", "txt")
        if os.path.exists(os.path.join(label_path,lb)):
            #画检测出来的
            draw_from_label(os.path.join(img_path,img),os.path.join(label_path,lb),os.path.join(save_path,img),label_name, draw_type=drawDefaultRectangle)
        if os.path.exists(os.path.join(gt_path,lb)):
            #画gt框
            draw_from_label(os.path.join(save_path,img),os.path.join(gt_path,lb),os.path.join(save_path,img), label_name)
